In [67]:
import pandas as pd
import numpy as np
fund_df = pd.read_csv('/Users/wenjingli/Desktop/python_data_analysis/大作业5/fund_150270 copy.csv',  index_col=0, parse_dates=['trade_date'])
fund_df.head()
fund_df.columns
fund_df_final = fund_df.loc[:,['ts_code','close_adj']]
fund_df_final.index

DatetimeIndex(['2016-08-01', '2016-08-02', '2016-08-03', '2016-08-04',
               '2016-08-05', '2016-08-08', '2016-08-09', '2016-08-10',
               '2016-08-11', '2016-08-12',
               ...
               '2020-08-31', '2020-09-01', '2020-09-02', '2020-09-03',
               '2020-09-04', '2020-09-07', '2020-09-08', '2020-09-09',
               '2020-09-10', '2020-09-11'],
              dtype='datetime64[ns]', name='trade_date', length=995, freq=None)

In [75]:

#智能定投， T-1复权价与历史均线比较： 高位少买，低位多买
# 若T-1收盘价高于500日平均值
# 实际扣款率
# 0-15%90%
# 15-50%80%
# 50-100%70%
# 100%以上60%

# 若T-1收盘价低于500日平均值
# 实际扣款率
# 0-5%160%
# 5-10%170%
# 10-20%180%
# 20-30%190%
# 30-40%200%
# 40%以上210%

def intelligent_periodic_investment(df, ma=500):
    
    df_ma = df['close_adj'].rolling(ma, min_periods=1).mean()
    bins = [0, 0.6, 0.7, 0.8, 0.9, 0.95, 1, 1.15,1.5, 2,1000]
    labels = 2.1,2,1.9, 1.8, 1.7, 1.6,  0.9, 0.8,0.7,0.6

    close_ma = df['close_adj']/df_ma 
    invest_ratio = pd.cut(close_ma, bins = bins, labels = labels)
    df['ma'] = df_ma
    df['close_ma'] =close_ma
    df['invest_ratio'] = invest_ratio.astype(float)
intelligent_periodic_investment(fund_df_final)

In [81]:
def calc_return(df, fixed_investment=100):
     # 计算累积投资份额、累计投资金额
    df[ 'fixed_investment'] = fixed_investment * df['invest_ratio']
    df['cum_investment'] = df['fixed_investment'].cumsum()
    df['buy_num'] = df['fixed_investment'] / df['close_adj']
    df['cum_buy_num'] = df['buy_num'].cumsum()
     # na值处理
    df[['cum_investment', 'cum_buy_num']] = df[['cum_investment', 'cum_buy_num']].ffill()
    df[['fixed_investment', 'buy_num']] = df[['fixed_investment', 'buy_num']].fillna(0)
     # 计算持有基金价值
    df['hold_fund_value'] = df['cum_buy_num']*df['close_adj']
    df['average_cost'] = df['cum_investment']/df['cum_buy_num']
    # 计算收益率
    df['return'] = df['hold_fund_value'] / df['cum_investment']
    return df[['cum_investment', 'cum_buy_num', 'hold_fund_value', 'average_cost', 'return']]
calc_return(fund_df_final)

,cum_investment,cum_buy_num,hold_fund_value,average_cost,return
trade_date,,,,,
2016-08-01,160.0,171.306210,160.000000,0.934000,1.000000
2016-08-02,320.0,343.534412,319.143469,0.931493,0.997323
2016-08-03,480.0,517.258625,476.395194,0.927969,0.992490
2016-08-04,640.0,690.606404,637.429711,0.926722,0.995984
2016-08-05,800.0,867.012137,786.380008,0.922709,0.982975
...,...,...,...,...,...
2020-09-07,87200.0,47833.811863,487489.300842,1.822978,5.590474
2020-09-08,87260.0,47840.010261,463087.472126,1.823996,5.306985
2020-09-09,87320.0,47846.580563,436934.973704,1.825000,5.003836


In [82]:
fund_df_final.head()

,ts_code,close_adj,ma,close_ma,invest_ratio,fixed_investment,cum_investment,buy_num,cum_buy_num,hold_fund_value,average_cost,return
trade_date,,,,,,,,,,,,
2016-08-01,150270.SZ,0.934,0.93400,1.000000,1.6,160.0,160.0,171.306210,171.306210,160.000000,0.934000,1.000000
2016-08-02,150270.SZ,0.929,0.93150,0.997316,1.6,160.0,320.0,172.228202,343.534412,319.143469,0.931493,0.997323
2016-08-03,150270.SZ,0.921,0.92800,0.992457,1.6,160.0,480.0,173.724213,517.258625,476.395194,0.927969,0.992490
2016-08-04,150270.SZ,0.923,0.92675,0.995954,1.6,160.0,640.0,173.347779,690.606404,637.429711,0.926722,0.995984
2016-08-05,150270.SZ,0.907,0.92280,0.982878,1.6,160.0,800.0,176.405733,867.012137,786.380008,0.922709,0.982975


In [85]:
# 2020-08 information
fund_df_final['2020-08']['fixed_investment'].sum()


<ipython-input-85-635bb28e9d79>:1: FutureWarning: Indexing a DataFrame with a datetimelike index using a single string to slice the rows, like `frame[string]`, is deprecated and will be removed in a future version. Use `frame.loc[string]` instead.
  fund_df_final['2020-08']['fixed_investment'].sum()


1200.0

In [90]:
fund_df_final['2020-08']['buy_num'].sum()

<ipython-input-90-0d81b0be4d94>:1: FutureWarning: Indexing a DataFrame with a datetimelike index using a single string to slice the rows, like `frame[string]`, is deprecated and will be removed in a future version. Use `frame.loc[string]` instead.
  fund_df_final['2020-08']['buy_num'].sum()


145.42636374146164